In [18]:
from pyomo.environ import *
from pyomo.opt import SolverFactory
import pyomo.environ
import pandas as pd
import numpy as np

In [2]:
TAs_df = pd.read_csv("data/TA_apps.csv").set_index("name")
str_to_list = lambda s: list(map(int,s.split(",")))
TAs_df["can_teach"   ] = TAs_df["can_teach"  ].apply(str_to_list)
TAs_df["enthusiastic"] = TAs_df["enthusiastic"].apply(str_to_list)
TAs = TAs_df.index.values.tolist()
TAs_df

,availability,R_proficiency,python_proficiency,can_teach,enthusiastic
name,,,,,
Alice,MTW,competent,expert,"[542, 551, 552]","[511, 571, 513]"
Bob,MTWR,competent,none,"[551, 523, 531, 552, 561]","[551, 523, 561]"
Carol,MT,expert,expert,"[542, 551, 523, 552, 571, 513]","[551, 552, 571]"
Chuck,MW,expert,beginner,"[511, 551, 531, 561]","[521, 542, 523]"
Dan,MTR,expert,expert,"[511, 521, 512, 552, 532, 571, 513]","[521, 512, 552]"
Erin,R,expert,expert,"[512, 532, 513]","[512, 513]"
Faith,MTWR,competent,expert,"[511, 521, 523, 531, 512, 561, 571, 513]","[521, 512, 513]"
Grace,MR,beginner,expert,"[521, 523, 512]",[511]
Heidi,TWR,expert,competent,[531],"[511, 551, 523, 552, 561, 532, 571]"


### For Availability:

M : Monday

T : Tuesday

W : Wednesday

R : Thursday

In [3]:
courses_df = pd.read_csv("data/MDS_courses_term_1.csv").set_index("course_number")
courses = courses_df.index.values.tolist()
blocks = set(courses_df["block"])
courses_df

,course_title,slug,block,primary_lang,lab_days
course_number,,,,,
511,Programming for Data Science,prog-dsci,1,both,WR
521,Computing Platforms for Data Science,platforms-dsci,1,both,MT
542,Communication and Argumentation,comm-arg,1,none,WR
551,Descriptive Statistics and Probability for Dat...,stat-prob-dsci,1,none,MT
512,Algorithms and Data Structures,alg-data-struct,2,python,WR
523,Data Wrangling,data-wrangling,2,R,WR
531,Data Visualization I,viz-1,2,both,MT
552,Statistical Inference and Computation I,stat-inf-1,2,R,MT
513,Databases and Data Retrieval,database-data-retr,3,both,WR


Goal: match TAs to courses so that our staffing needs are covered in the best way possible.

Here are the **constraints**:

- Each course should be assigned to exactly 2 TAs.
- A TA can only cover one course at a time (i.e., in a given block).
- A TA can only be assigned to a course they have listed as "can teach" or "enthusiastic to teach".
- To cover a course, the TA must be available for one of the two lab days (for simplicity).

And here is the **objective**:

- We want to maximize the number of assigned courses that TAs are enthusiastic about.

In [4]:
model = ConcreteModel()

## Define sets ##
#  Sets
#       i   TAs  / Alice, Bob /
#       j   course_number          / 511, 521 / ;
model.i = Set(initialize=TAs, doc='TAs')
model.j = Set(initialize=courses, doc='course_number')


- For this problem, decision variables are best handled with _binary variables_.

Let $x_{ij}$ be $1$ if TA $i$ is assigned to course $j$, and 0 otherwise.

In [20]:
# Sample Output

x_df = pd.DataFrame(data=(np.random.rand(len(TAs),len(courses))<.2).astype(int), index=TAs, columns=courses)
x_df

,511,521,542,551,512,523,531,552,513,561,571,532
Alice,0,0,0,0,1,1,0,0,0,1,0,0
Bob,0,1,0,0,0,0,0,0,0,0,0,0
Carol,1,0,0,0,1,0,1,0,0,0,0,1
Chuck,0,1,0,0,0,0,0,0,1,0,0,1
Dan,1,1,0,0,0,0,0,0,0,1,0,1
Erin,0,0,1,1,1,1,0,1,1,1,0,0
Faith,1,1,0,0,0,0,0,0,0,0,1,0
Grace,0,0,0,0,0,0,0,0,1,0,0,0
Heidi,0,0,0,0,0,0,0,1,0,1,1,0
Ivan,1,0,0,0,0,0,0,1,0,1,0,0


In [5]:
# Variable of 2 dimensions 

model.x = Var(model.i, model.j, doc='TA Assignment', within=Binary)

#### Setting Up Constraints



_Each course should be assigned exactly 2 TAs._
  - For all courses $j$, we require $\sum_i x_{ij} = 2$
  - That is, the sum of TAs for DSCI 511 equals 2, the sum of TAs for DSCI 521 equals 2, ...


In [6]:
TAS_PER_COURSE = 2
model.limits = ConstraintList()
for course in courses:
    model.limits.add(expr = sum(model.x[TA, course] for TA in TAs) == TAS_PER_COURSE)

_A TA can only cover one course at a time (i.e., in a given block)._
  - For all TAs $i$, for all blocks, we require $\sum_j x_{ij} \leq 1$
  - That is, the sum of Alice's Block 1 courses is at most 1, the sum of Bob's Block 1 courses is at most 1, ...

In [7]:
for TA in TAs:
    for block in blocks:
        model.limits.add(expr = sum(model.x[TA, course] for course in courses if courses_df.loc[course]["block"] == block) <=1)

_A TA can only be assigned to a course they have listed as "can teach" or "enthusiastic to teach"._
  - For all $i,j$ such that canteach$(i,j)$ is false and enthusiastic$(i,j)$ is false, $x_{ij}=0$
  - That is, Bob cannot teach 511 because canteach(Bob,511) is false and enthusiastic(Bob,511) is false, ...
 

In [8]:
for TA in TAs:
    for course in courses:
        if course not in TAs_df.loc[TA]["can_teach"] and course not in TAs_df.loc[TA]["enthusiastic"]:
            model.limits.add(model.x[TA, course] == 0)

_To cover a course, the TA must be available for one of the two lab days._
  - For all $i,j$ such that available$(i,\text{day1}(j))$ is false and available$(i,\text{day2}(j))$ is false, $x_{ij}=0$
  - That is, Carol cannot teach DSCI 542 because available(Carol,W) is false and available(Carol,Th) is false
    - And day1(542) is W, day2(542) is Th.
  

In [9]:
for TA in TAs:
    for course in courses:
        if courses_df.loc[course]["lab_days"][0] not in TAs_df.loc[TA]["availability"] and courses_df.loc[course]["lab_days"][1] not in TAs_df.loc[TA]["availability"]:
            model.limits.add(model.x[TA, course] == 0)

#### Setting up the objective

_We want to maximize the number of assigned courses that TAs are enthusiastic about._
  - Maximize the following objective: for all $i,j$ such that enthusiastic$(i,j)$ is true, $\sum_{ij} x_{ij}$
  
Another way of writing this:
  - Maximize $\sum_{ij} \text{enthusiastic}(i,j)x_{ij}$
  - This assumes the values of enthusiastic$(i,j)$ are either 0 or 1, so that the product 1 is only when the TA is enthusiastic and assigned to the course. 

In [10]:
model.objective = Objective(expr = sum(model.x[TA,course] for TA in TAs for course in courses if course in TAs_df.loc[TA]["enthusiastic"]),sense = maximize )


In [11]:
def pyomo_postprocess(options=None, instance=None, results=None):
    instance.x.display()

In [12]:
opt = SolverFactory("glpk")
results = opt.solve(model)

results.write()
print("\nDisplaying Solution\n" + '-' * 60)
pyomo_postprocess(None, model, results)

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 21.0
  Upper bound: 21.0
  Number of objectives: 1
  Number of constraints: 124
  Number of variables: 133
  Number of nonzeros: 343
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 0.030986547470092773
# ----------------------------------------------------------
#   Solution Information
# --------------------------------

In [13]:
print("We have %d enthusiastic courses out of a possible %d." % 
      (results.Problem()['Lower bound'], len(courses)*TAS_PER_COURSE))

We have 21 enthusiastic courses out of a possible 24.


#### Printing the results

In [14]:
out_df_by_TA = pd.DataFrame("", index=TAs, columns=blocks)
for TA in TAs:
    for course in courses:
        if model.x[TA, course].value == 1:
            out_df_by_TA.at[TA, courses_df.loc[course]["block"]] = course
out_df_by_TA

,1,2,3
Alice,511,,571
Bob,551,523,561
Carol,,552,571
Chuck,542,523,
Dan,521,512,
Erin,,512,513
Faith,521,531,513
Grace,,,
Heidi,551,552,532
Ivan,511,,532


In [15]:
for course in courses:
    print(course, end=": ")
    for TA in TAs:
        if model.x[TA, course].value == 1:
            print(TA, end=', ')
    print("")

511: Alice, Ivan, 
521: Dan, Faith, 
542: Chuck, Judy, 
551: Bob, Heidi, 
512: Dan, Erin, 
523: Bob, Chuck, 
531: Faith, Judy, 
552: Carol, Heidi, 
513: Erin, Faith, 
561: Bob, Judy, 
571: Alice, Carol, 
532: Heidi, Ivan, 


In [16]:
# Enthusiastic courses
for course in courses:
    for TA in TAs:
        if model.x[TA, course].value == 1 and course in TAs_df.loc[TA]["enthusiastic"]:
            print("%-7s is enthusiastic about DSCI %d!" % (TA, course))

Alice   is enthusiastic about DSCI 511!
Ivan    is enthusiastic about DSCI 511!
Dan     is enthusiastic about DSCI 521!
Faith   is enthusiastic about DSCI 521!
Chuck   is enthusiastic about DSCI 542!
Judy    is enthusiastic about DSCI 542!
Bob     is enthusiastic about DSCI 551!
Heidi   is enthusiastic about DSCI 551!
Dan     is enthusiastic about DSCI 512!
Erin    is enthusiastic about DSCI 512!
Bob     is enthusiastic about DSCI 523!
Chuck   is enthusiastic about DSCI 523!
Judy    is enthusiastic about DSCI 531!
Carol   is enthusiastic about DSCI 552!
Heidi   is enthusiastic about DSCI 552!
Erin    is enthusiastic about DSCI 513!
Faith   is enthusiastic about DSCI 513!
Bob     is enthusiastic about DSCI 561!
Alice   is enthusiastic about DSCI 571!
Carol   is enthusiastic about DSCI 571!
Heidi   is enthusiastic about DSCI 532!
